In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [4]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-large', dimensions=1024)

In [5]:
file_path = "../../2.1-DataIngestion/speech.txt"

In [6]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(file_path)

In [7]:
loader.load()

[Document(metadata={'source': '../../2.1-DataIngestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit 

In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)

In [9]:
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [10]:
split_docs = splitter.split_documents(loader.load())

In [11]:
split_docs

[Document(metadata={'source': '../../2.1-DataIngestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'),
 Document(metadata={'source': '../../2.1-DataIngestion/speech.txt'}, page_content='Just because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Doc

In [12]:
# Embedding

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

In [13]:
# create index and vectore store
index = faiss.IndexFlatL2(1024)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [14]:
split_docs[0].page_content

'The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.'

In [15]:
# Extract text from split_docs (a list of Document objects)
#texts = [doc.page_content for doc in split_docs]

# Now call embed_documents correctly
#embeddings.embed_documents(texts)

In [16]:
vector_store.add_documents(documents=split_docs)

['b10982c3-b1da-4760-ac12-2a965ae74993',
 '3a73b947-de35-4994-b656-73f7d447b192',
 '353e963e-52ee-45dc-a142-fba3bb91934c',
 '06aca659-a9bd-4a97-8007-80dcc0806701',
 'a72fcb9a-f5be-41b9-bee9-78b1619d99d1',
 '7614dbe6-69d0-4f59-b8bc-8c2f50c904aa',
 'ca1e7fac-5555-41a6-8c59-7f92087b77c3',
 '5be3fa67-39ad-4839-9824-d8e7a375ccf6',
 '7a230d25-ec2c-47f0-bbc8-3d66ce746358',
 'aa708094-77b4-407c-87ce-5e005c804fd4']

In [17]:
vector_store.similarity_search('What is democracy?',k=2)

[Document(id='7a230d25-ec2c-47f0-bbc8-3d66ce746358', metadata={'source': '../../2.1-DataIngestion/speech.txt'}, page_content='always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='b10982c3-b1da-4760-ac12-2a965ae74993', metadata={'source': '../../2.1-DataIngestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made

In [18]:
retriever=vector_store.as_retriever(search_kwargs={"k": 3})

In [19]:
retriever.invoke('What is democracy?')

[Document(id='7a230d25-ec2c-47f0-bbc8-3d66ce746358', metadata={'source': '../../2.1-DataIngestion/speech.txt'}, page_content='always carried nearest our hearts—for democracy, for the right of those who submit to authority to have a voice in their own governments, for the rights and liberties of small nations, for a universal dominion of right by such a concert of free peoples as shall bring peace and safety to all nations and make the world itself at last free.'),
 Document(id='b10982c3-b1da-4760-ac12-2a965ae74993', metadata={'source': '../../2.1-DataIngestion/speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made

In [20]:
#vector_store.save_local("today's class faiss index-practice")

In [21]:
# LLm

from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model='gpt-4.1')



In [22]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [23]:
def format_doc(docs):
    return "/n/n".join(doc.page_content for doc in docs)

In [24]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [27]:
rag_chain = (
    {"context": retriever | format_doc, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [28]:
rag_chain.invoke("What is Democracy?")

'Democracy is a system of government in which those who submit to authority have a voice in their own governments, ensuring political liberty and the rights and liberties of all people. It is rooted in the principle that the rights of individuals and nations should be secured by the collective will of free peoples. Ultimately, democracy seeks to create a foundation of peace, freedom, and universal rights.'